In [1]:
# In the name of Allah

In [87]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import keras.backend as K
import tensorflow as tf
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn.metrics import confusion_matrix
import joblib
import pickle

In [95]:
with open('X_train_real.pkl', 'rb') as f:
        X_train = pickle.load(f)
with open('y_train_real.pkl', 'rb') as f:
    y_train = pickle.load(f)

In [96]:
print(len(X_train))
print(len(y_train))
print(Counter(y_train))

532264
532264
Counter({0.0: 266132, 1.0: 266132})


In [89]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2,random_state=109) # 70% training a

In [90]:
print(len(X_train),len(y_train))
print(len(X_test),len(y_test))
print(Counter(y_train),Counter(y_test))

60 60
16 16
Counter({0.0: 58, 1.0: 2}) Counter({0.0: 15, 1.0: 1})


## Hyperparameter Tunning :- Random Forest Classifier

In [46]:
clf=RandomForestClassifier(n_estimators=10)

In [47]:
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

In [48]:
g_search = GridSearchCV(estimator = clf, param_grid = param_grid, 

                          cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

In [50]:
g_search.fit(X_train, y_train);

print(g_search.best_params_)

{'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 8}


In [51]:
clf=g_search.best_estimator_

In [52]:
print('Best parameters:', g_search.best_params_)
print('Best CV score:', g_search.best_score_)
#print('Model size: {:0.2f}M'.format(g_search.best_estimator_.size_ / 1000000))

Best parameters: {'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 8}
Best CV score: 0.9738461538461539


In [91]:
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[15,  0],
       [ 1,  0]])

In [ ]:
# Save the model as a pickle in a file
joblib.dump(clf, 'clf_model_real.pkl')

## HyperParameter Tunning :- SVM Classifier

In [49]:
model = SVC()
model.fit(X_train, y_train)
  
# print prediction results
#predictions = model.predict(X_test)
#print(classification_report(y_test, predictions))

SVC()

In [50]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

In [54]:
g_search = GridSearchCV(estimator = SVC(), param_grid = param_grid, 

                          cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

In [55]:
g_search.fit(X_train, Y_train);

#print(g_search.best_params_)

In [56]:
model=g_search.best_estimator_

In [57]:
print('Best parameters:', g_search.best_params_)
print('Best CV score:', g_search.best_score_)

Best parameters: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
Best CV score: 0.957997311827957


In [93]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 2, 13],
       [ 1,  0]])

In [ ]:
# Save the model as a pickle in a file
joblib.dump(model, 'svm_model_real.pkl')

## Hyper Parameter Tunning :- ANN

In [23]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [24]:
# define the keras model
def create_model(neurons1=1,neurons2=1,neurons3=1):
    model = Sequential()
    model.add(Dense(neurons1, input_dim=30, activation='relu'))
    model.add(Dense(neurons2, activation='relu'))
    model.add(Dense(neurons3, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])
    return model

In [25]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=32, verbose=0)

In [26]:
neurons1 = [6, 8, 10, 12, 14, 16]
neurons2 = [4, 6, 8, 10, 12]
neurons3 = [4,5,6,7]

# neurons1 = [6, 8]
# neurons2 = [4, 6]
# neurons3 = [4,5]
param_grid = {
    'neurons1' : neurons1,
    'neurons2' : neurons2,
    'neurons3' : neurons3
    }
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid_search.fit(X_train, Y_train)

In [28]:
model=grid_search.best_estimator_

In [29]:
print('Best parameters:', grid_search.best_params_)
print('Best CV score:', grid_search.best_score_)

Best parameters: {'neurons1': 8, 'neurons2': 4, 'neurons3': 4}
Best CV score: 0.6764112909634908


In [94]:
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[15,  0],
       [ 1,  0]])

In [ ]:
# Save the model as a pickle in a file
joblib.dump(model, 'ann_model_real.pkl')